In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("datasets_lib/mnist/",one_hot=True)


Extracting datasets_lib/mnist/train-images-idx3-ubyte.gz
Extracting datasets_lib/mnist/train-labels-idx1-ubyte.gz
Extracting datasets_lib/mnist/t10k-images-idx3-ubyte.gz
Extracting datasets_lib/mnist/t10k-labels-idx1-ubyte.gz


In [2]:
# Traingin params:
learning_rate = 0.001
training_steps = 10000
batch_size = 128
display_step = 200

# Hyper params
num_input = 28
timesteps = 28 #[each 28 * 28 image is handled as 28 timesteps of a sequence of size 28 ...cheers !!!]
num_hidden = 128 #hidden layer number of features.
num_classes = 10 #MNIST total classes (0-9)

# input feature and label 

X = tf.placeholder(tf.float32,[None,timesteps,num_input])
Y = tf.placeholder(tf.float32,[None,num_classes])


In [3]:
# Now let's define the weights : Notice here that the total weight matrix size,
#will be double the row, as we have to take into consideration forward and backward cells

weights = tf.get_variable("weight",dtype=tf.float32,initializer=tf.random_normal_initializer(),
                           shape=[2*num_hidden,num_classes])

bias = tf.get_variable("bias",dtype=tf.float32,initializer=tf.random_normal_initializer(),shape=[num_classes])

print tf.shape(bias)

Tensor("Shape:0", shape=(1,), dtype=int32)


In [4]:
# Now we define our bidirectional RNN
def BiLSTM(x,weights,biases):
    # First we have to model data in the way RNN input wants. 
    # Current data input shape: (batch_size, timesteps, n_input)
    # Required shape: 'timesteps' tensors list of shape (batch_size, num_input)
    
    # This can be done using unpack / unstack operation in TF
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, num_input)
    x = tf.unstack(x,timesteps,1)
    
    # Define lstm cells with tensorflow
    
    lstm_fwd_cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden,forget_bias=1.0)
    lstm_bw_cell = tf.nn.rnn_cell.BasicLSTMCell(num_hidden,forget_bias=1.0)
    
    outputs, _ ,_ = tf.nn.static_bidirectional_rnn(lstm_fwd_cell,lstm_bw_cell,x,dtype=tf.float32)
    print tf.shape(outputs)
    return tf.matmul(outputs[0],weights)+biases
    #return outputs
    
    

In [5]:
logits = BiLSTM(X,weights,bias)

prediction = tf.nn.softmax(logits)

#Define loss and optimizer

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=Y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)


correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()


Tensor("Shape_1:0", shape=(3,), dtype=int32)


In [7]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    for step in range(1, training_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Reshape data to get 28 seq of 28 elements
        batch_x = batch_x.reshape((batch_size, timesteps, num_input))
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

    print("Optimization Finished!")

    # Calculate accuracy for 128 mnist test images
    test_len = 128
    test_data = mnist.test.images[:test_len].reshape((-1, timesteps, num_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: test_data, Y: test_label}))

Step 1, Minibatch Loss= 2.8383, Training Accuracy= 0.055
Step 200, Minibatch Loss= 2.2243, Training Accuracy= 0.180
Step 400, Minibatch Loss= 1.9548, Training Accuracy= 0.414
Step 600, Minibatch Loss= 1.8308, Training Accuracy= 0.469
Step 800, Minibatch Loss= 1.7383, Training Accuracy= 0.508
Step 1000, Minibatch Loss= 1.6879, Training Accuracy= 0.484
Step 1200, Minibatch Loss= 1.5472, Training Accuracy= 0.578
Step 1400, Minibatch Loss= 1.4674, Training Accuracy= 0.500
Step 1600, Minibatch Loss= 1.3338, Training Accuracy= 0.594
Step 1800, Minibatch Loss= 1.2088, Training Accuracy= 0.656
Step 2000, Minibatch Loss= 1.4087, Training Accuracy= 0.539


KeyboardInterrupt: 